In [1]:
# import optuna
# from optuna.trial import Trial
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.metrics import AUC
from matplotlib import pyplot

In [2]:
# Data Loading
print("Loading data...")
X_model = np.load('../../data/X_timeseries_expanded.npy')
Y_model = pd.read_csv('../../data/Y_model.csv').values

Loading data...


In [3]:
# Train Test Split
print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(X_model, Y_model, test_size=0.2, random_state=42)

Splitting data...


In [4]:
# design network
# References: 
# - https://towardsdatascience.com/choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras-f8e9ed76f046
# - https://stats.stackexchange.com/questions/242238/what-is-considered-a-normal-quantity-of-outliers
# - https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
hidden_nodes = int(2/3 * (X_train.shape[1] * X_train.shape[2]))
model = Sequential()
model.add(LSTM(hidden_nodes, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('softmax'))
model.compile(
    loss='mae',
    optimizer='adam',
    metrics=[AUC()]
)

2022-12-14 23:54:53.717026: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-14 23:54:53.718085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [5]:
print("Training model...")
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=100,
    validation_data=(X_test, y_test), 
    verbose=2,
    shuffle=False
)

Training model...


2022-12-14 23:55:18.877755: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-14 23:55:18.884069: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/50


2022-12-14 23:55:19.516838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-14 23:55:19.654007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-14 23:55:20.864158: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


: 

: 

In [ ]:
# plot history
pyplot.plot(history.history['auc'], label='train')
pyplot.plot(history.history['val_auc'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# Save model with pickle
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)